In [4]:
import faiss
import pickle
import numpy as np
import gradio as gr
from crewai import Agent, Task, Crew,LLM
import os

In [2]:
from sentence_transformers import SentenceTransformer
EMBEDDING_MODEL = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

C:\Users\aravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
os.environ["GEMINI_API_KEY"] = ""
llm = "gemini/gemini-1.5-flash"
llm = LLM(model=llm)


In [8]:
faiss_index_path = 'D:\\New folder\\faiss_index.bin'
dict_path = 'D:\\New folder\\dictionary.pkl'

In [9]:
vector_db = faiss.read_index(faiss_index_path)
with open(dict_path, "rb") as file:
    doc_store = pickle.load(file)

In [34]:

class ChatCrew:
    def __init__(self, EMBEDDING_MODEL, vector_db, doc_store):
        # Define the retrieval agent
        self.retrieval_agent = Agent(
            role="AI Research Assistant",
            goal="Retrieve and provide the most relevant information based on user queries.",
            backstory="You are an AI-powered assistant designed to search through a database and provide users with the best-matching responses.",
            llm=llm,
            verbose=True
        )
        self.EMBEDDING_MODEL = EMBEDDING_MODEL
        self.vector_db = vector_db
        self.doc_store = doc_store

    def retrieve_answer(self, query):
        """Retrieve the most relevant answer from FAISS"""
        # Convert query to vector
        query_embedding = np.array(
            self.EMBEDDING_MODEL.encode([query], convert_to_numpy=True, normalize_embeddings=True), 
            dtype=np.float32
        )
    
        # Search in FAISS
        _, indices = self.vector_db.search(query_embedding, k=3)  # Retrieve top 3 results
    
        # Retrieve context from document store
        context = [self.doc_store[idx] for idx in indices[0] if idx in self.doc_store]
    
        # Format response
        return "\n\n".join(context) if context else "No relevant information found."

    def create_chatbot_crew(self, query):
        context = self.retrieve_answer(query)
        query_task = Task(
            description=f"""Process user query and retrieve the best response. Do not answer anything beyond the retrieved context.
                             The query is: {query} and context is {context}
            """,
            agent=self.retrieval_agent,
            expected_output="The most relevant answer retrieved from the knowledge base.",
            verbose=True
        )

        # Define Crew
        self.chatbot_crew = Crew(
            agents=[self.retrieval_agent],
            tasks=[query_task],
            verbose=True
        )

    def chatbot_interface(self, query):
        self.create_chatbot_crew(query)
        return self.chatbot_crew.kickoff()

# Create instance of ChatCrew
chatbot = ChatCrew(EMBEDDING_MODEL, vector_db, doc_store)

demo = gr.Interface(fn=chatbot.chatbot_interface, inputs="text", outputs="text")
demo.launch()


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


# Agent: AI Research Assistant
## Task: Process user query and retrieve the best response. Do not answer anything beyond the retrieved context.
                             The query is: How to check Airtel DTH balance? and context is The Code has to be sent to 54325 from your registered mobile
7) How to check Airtel DTH balance?
You can check the Airtel DTH Balance in the following 3 ways:
SMS ‘BAL’ to 54325 from your Registered Mobile Number
From your registered mobile give a missed call to 81300-81300
On the Airtel Thanks App on the Services page, the DTH account has the balance
8) Which Is The Best DTH HD Connection?
Airtel is considered the best DTH HD connection in India by customers and industry experts because of the cutting-edge features that are
included in their offerings. Another reason to make Airtel DTH the best in the market is the attractive packages and pricing of its packs.

Enter the recharge amount or browse for plans
Confirm and pay through Airtel Payments Bank, De